In [1]:
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

In [2]:
def sleep(x):
    time.sleep(x)
    
def wait(x):
    driver.implicitly_wait(x)
    
def click_bann_byID(ID):
    actions = ActionChains(driver)
    akzeptieren = driver.find_element(By.ID, ID)
    actions.click(akzeptieren).perform()
    wait(10)
    sleep(0.5)

def find_elements_HPCO(H,P,C,O):
    if website_name == 'jobware':
        header = driver.find_elements(By.TAG_NAME, H)
    else:
        header = driver.find_elements(By.CLASS_NAME, H)
    publish = driver.find_elements(By.CLASS_NAME, P)
    company = driver.find_elements(By.CLASS_NAME, C)
    ort = driver.find_elements(By.CLASS_NAME, O) 

    list_header = [title.text for title in header]
    list_publish = [pub.text for pub in publish]
    list_company = [comp.text for comp in company]
    list_ort = [o.text for o in ort]
    return list_header, list_publish, list_company, list_ort
    
def scroll_down(x):
    n=0
    while n < x:
        n+=1
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
        wait(10)
        sleep(2)

# 01 - STEPSTONE

In [3]:
'''
Title : Web Scrapping by Selenium 
Project Purpose: From StepStone scrap data for some Job Titels
1 - Create Driver
2 - Go to Website
3 - Create ActionChain Object
    3.1 - Click Banned Accept
4 - Take Title and Infos from Page
    4.1 - Create Lists 
    4.2 - Create DataFrame
    4.3 - Repeat Process
    4.4 - Print and Save DataFrame
'''

print('---------------------- StepStone Job Searching Selenium Project ----------------------')
start=datetime.now()  
# 0 Link Descriptions
link_original_stepstone = 'https://www.stepstone.de/jobs/data-analyst/in-rietberg?radius=50&page=2'

website_name = 'stepstone'
job_name = 'Data Analyst'
ort_ = 'Rietberg'
radius = 50
page_number = 1

#  1 - Create Driver
Path = '/Users/macbook/Desktop/projects/Github_Repositories/Portfolio Projects/02 - Web_Scraping_Job_Search/chromedriver'
driver = webdriver.Chrome(Path)

#  2 - Go to Website
job_link = job_name.replace(' ', '-').lower()
ort_link = ort_.lower()
link = f'https://www.stepstone.de/jobs/{job_link}/in-{ort_link}?radius={radius}&page={page_number}'

driver.get(link)
wait(10)
sleep(2)

#  3 - ActionChain Object created
# 3.1 - Click Banned Accept
ID = 'ccmgt_explicit_accept'
click_bann_byID(ID)

# 4 -  Take Infos from Page
# Headers, Publish_Time ,Company, City
H, P, C, O = 'resultlist-12iu5pk', 'resultlist-3asi6i', 'resultlist-1v262t5', 'resultlist-dettfq'
list_header, list_publish, list_company, list_ort = find_elements_HPCO(H,P,C,O)

# Description and Page number of results
description = driver.find_elements(By.CLASS_NAME, 'resultlist-1pq4x2u')
result = driver.find_elements(By.CLASS_NAME, 'resultlist-xeyevn')

# 4.1 - Get Texts for each finding
list_description = [des.text for des in description]
print('Header',len(list_header), 'Publish',len(list_publish), 'Company',len(list_company), 'Ort',len(list_ort), 'desc', len(list_description))

# Total Search Page Number
list_result = [res.text for res in result]
number_of_page = int(list_result[0].split(' ')[-1])
print(f'Number of Jobs Pages = {number_of_page}')

# 4.2 - DataFrame df
d = dict(job_title=np.array(list_header), publish=np.array(list_publish), company_name=np.array(list_company), city=np.array(list_ort) , description=np.array(list_description))
df = pd.DataFrame.from_dict(d, orient='index')
df = df.T


# 4.3 Repeat Process for every Web Page
while  page_number < number_of_page:
    page_number+=1
    
    # 4.1 - Go to another page
    link = f'https://www.stepstone.de/jobs/{job_link}/in-{ort_link}?radius={radius}&page={page_number}'
    driver.get(link)
    wait(10)
    sleep(1.5)
    
    # 4.2 - Find the elements and get the Texts
    list_header, list_publish, list_company, list_ort = find_elements_HPCO(H,P,C,O) 
    description = driver.find_elements(By.CLASS_NAME, 'resultlist-1pq4x2u')
    list_description = [des.text for des in description]
 
    # 4.3 - Create new page Dataframe
    d = dict(job_title=np.array(list_header), publish=np.array(list_publish), company_name=np.array(list_company), city=np.array(list_ort) , description=np.array(list_description))
    df2 = pd.DataFrame.from_dict(d, orient='index')
    df2 = df2.T
    
    # 4.4 - Concatenate the DataFrames
    df = pd.concat([df,df2], axis=0, ignore_index=True)
    print(f'Page Number : {page_number}, DataFrame Shape : {df2.shape}')
    

# 4.4 Save Data as csv and xlsx    
print(f'DataFrame End : {df.shape}')
df['website'] = website_name
# 4.3 - Save DataFrame
# 4.3.1 - to csv
path = '/Users/macbook/Desktop/projects/Github_Repositories/Portfolio Projects/02 - Web_Scraping_Job_Search/data'
job_name2 = job_name.replace(' ', '-')
time_ = datetime.today().strftime('%Y-%m-%d')
df.to_csv(f'{path}/{job_name2}-{time_}.csv', index=False)

end =datetime.now() 
print('Code Runned No Problem')
print(f'Time = {end - start}')
sleep(5)
driver.quit()

---------------------- StepStone Job Searching Selenium Project ----------------------
Header 25 Publish 25 Company 25 Ort 25 desc 25
Number of Jobs Pages = 22
Page Number : 2, DataFrame Shape : (25, 5)
Page Number : 3, DataFrame Shape : (25, 5)
Page Number : 4, DataFrame Shape : (25, 5)
Page Number : 5, DataFrame Shape : (25, 5)
Page Number : 6, DataFrame Shape : (25, 5)
Page Number : 7, DataFrame Shape : (25, 5)
Page Number : 8, DataFrame Shape : (25, 5)
Page Number : 9, DataFrame Shape : (25, 5)
Page Number : 10, DataFrame Shape : (25, 5)
Page Number : 11, DataFrame Shape : (25, 5)
Page Number : 12, DataFrame Shape : (25, 5)
Page Number : 13, DataFrame Shape : (25, 5)
Page Number : 14, DataFrame Shape : (25, 5)
Page Number : 15, DataFrame Shape : (25, 5)
Page Number : 16, DataFrame Shape : (25, 5)
Page Number : 17, DataFrame Shape : (25, 5)
Page Number : 18, DataFrame Shape : (25, 5)
Page Number : 19, DataFrame Shape : (25, 5)
Page Number : 20, DataFrame Shape : (25, 5)
Page Number 

# 02 - JOBWARE

In [4]:
print('---------------------- Jobware Job Searching Selenium Project ----------------------')

start=datetime.now()  
# 0 Link Descriptions
link_original = 'https://www.jobware.de/jobsuche?jw_jobname=data%20analyst&jw_jobort=333**%20Rietberg&jw_ort_distance=50'

website_name = 'jobware'
job_name = 'Data Analyst'
ort_ = 'Rietberg'
radius = 50
page_number = 0

#  1 - Create Driver
Path = '/Users/macbook/Desktop/projects/Github_Repositories/Portfolio Projects/02 - Web_Scraping_Job_Search/chromedriver'
driver = webdriver.Chrome(Path)

#  2 - Go to Website
job_link = job_name.replace(' ', '%20').lower()
ort_link = ort_.capitalize()
link = f'https://www.jobware.de/jobsuche?jw_jobname={job_link}&jw_jobort=333**%20{ort_}&jw_ort_distance={radius}'

driver.get(link)
wait(10)
sleep(2)

#  3 - ActionChain Object created
# 3.1 - Click Banned Accept
actions = ActionChains(driver)
akzeptieren = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div[2]/button')
actions.click(akzeptieren).perform()
wait(10)
sleep(0.5)
#dsgvo-1B76C4DA4B-orange dsgvo-1B76C4DA4B-accept

# 4 -  Take Infos from Page
# Headers, Company, City, Description
H, P, C, O = 'h2', 'date', 'company', 'location'
list_header, list_publish, list_company, list_ort = find_elements_HPCO(H,P,C,O)
description = driver.find_elements(By.CLASS_NAME, 'task')
list_description = [des.text for des in description]

print('Header',len(list_header), 'Publish',len(list_publish), 'Company',len(list_company), 'Ort',len(list_ort), 'Desc', len(list_description))

# Total Search Page Number
result = driver.find_elements(By.CLASS_NAME, 'result-sort')
list_result = [res.text for res in result]
print(list_result)

# 4.2 - DataFrame df
d = dict(job_title=np.array(list_header), publish=np.array(list_publish), company_name=np.array(list_company), city=np.array(list_ort) , description=np.array(list_description))
df = pd.DataFrame.from_dict(d, orient='index')
df = df.T

# 4.4 Save Data as csv and xlsx    
print(f'DataFrame End : {df.shape}')
df['website'] = website_name
# 4.3 - Save DataFrame
# 4.3.1 - to csv
df.to_csv(f'{path}/{job_name2}-{time_}.csv', mode='a', index=False, header=False)

end =datetime.now() 
print('Code Runned No Problem')
print(f'Time = {end - start}')
sleep(5)
driver.quit()

---------------------- Jobware Job Searching Selenium Project ----------------------
Header 12 Publish 12 Company 12 Ort 12 Desc 11
['12 Treffer\nSortierung: Relevanz - Datum']
DataFrame End : (12, 5)
Code Runned No Problem
Time = 0:00:12.610382


# 03 - LINKEDIN (Chrome)

In [5]:
print('---------------------- Linkedin Job Searching Selenium Project ----------------------')
def sleep(x):
    time.sleep(x)
def wait(x):
    driver.implicitly_wait(x)
    

    #sleep(1.5)
    #more_option = driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button')
    #actions.click(more_option).perform()
    #wait(10)
    #sleep(0.5)    
    

start=datetime.now()  
# 0 Link Descriptions
link_original = 'https://www.linkedin.com/jobs/search/?currentJobId=3199974140&distance=25&keywords=data%20analyst&location=Rietberg' 

website_name =  'linkedin'
radius = 40
page_number = 1

#  1 - Create Driver
Path = '/Users/macbook/Desktop/projects/Github_Repositories/Portfolio Projects/02 - Web_Scraping_Job_Search/chromedriver'
driver = webdriver.Chrome(Path)

#  2 - Go to Website
job_link = job_name.replace(' ', '%20').lower()

link2 = f'https://www.linkedin.com/jobs/search/?distance=25&keywords={job_link}&location={ort_}'
driver.get(link2)
wait(10)
sleep(2)


#  3 - ActionChain Object created
# 3.1 - Click Banned Accept
actions = ActionChains(driver)
akzeptieren = driver.find_element(By.TAG_NAME, 'button')
actions.click(akzeptieren).perform()
wait(10)
sleep(0.5)

# 3.1 - 
scroller = driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button')

scroll_down(7)

# 4 -  Take Infos from Page
# Headers, Company, City, Description
header = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
publish = driver.find_elements(By.CLASS_NAME, 'job-search-card__listdate')
company = driver.find_elements(By.CLASS_NAME, 'hidden-nested-link')
ort = driver.find_elements(By.CLASS_NAME, 'job-search-card__location') 
#description = driver.find_elements(By.CLASS_NAME, 'resultlist-1pq4x2u')
result = driver.find_elements(By.CLASS_NAME, 'results-context-header__context')

# 4.1 -
list_header = [title.text for title in header]
list_publish = [pub.text for pub in publish]
list_company = [comp.text for comp in company]
list_ort = [o.text for o in ort]
#list_description = [des.text for des in description]

print('Header',len(list_header), 'Publish',len(list_publish), 'Company',len(list_company), 'Ort',len(list_ort))

# Total Search Page Number
list_result = [res.text for res in result]
print(f'Number of Jobs Pages = {list_result}')

# 4.2 - DataFrame df
d = dict(job_title=np.array(list_header), publish=np.array(list_publish), company_name=np.array(list_company), city=np.array(list_ort))
df = pd.DataFrame.from_dict(d, orient='index')
df = df.T
df['description'] = None
df['website'] = website_name

# 4.4 Save Data as csv and xlsx    
print(f'DataFrame End : {df.shape}')
# 4.3 - Save DataFrame
# 4.3.1 - to csv
df.to_csv(f'{path}/{job_name2}-{time_}.csv', mode='a', index=False, header=False)

# 4.3.2 - to excel
# install openpyxl
#df.to_excel(f'{path}/{job_name2}-{time_}.xlsx', sheet_name='Sheet3')

end =datetime.now() 
print('Code Runned No Problem')
print(f'Time = {end - start}')
sleep(5)
driver.quit()

df.head()

---------------------- Linkedin Job Searching Selenium Project ----------------------
Header 175 Publish 170 Company 162 Ort 175
Number of Jobs Pages = ['Rietberg, Kuzey Ren-Vestfalya, Almanya konumunda 847 Data Analyst iş ilanı (20 yeni)']
DataFrame End : (175, 6)
Code Runned No Problem
Time = 0:00:38.961719


,job_title,publish,company_name,city,description,website
0,Data Analyst (w/m/d) Power BI mit Remote-Anteil,1 hafta önce,ATLAS TITAN Mitte GmbH,"Gütersloh, Kuzey Ren-Vestfalya, Almanya",None,linkedin
1,Data Analyst / Scientist (m/w/d),2 hafta önce,Dr. Wolff Group,"Bielefeld, Kuzey Ren-Vestfalya, Almanya",None,linkedin
2,Data / Business Analyst (m/w/d),2 hafta önce,Dr. Wolff Group,"Bielefeld, Kuzey Ren-Vestfalya, Almanya",None,linkedin
3,CRM Data Analyst (m/w/d),1 gün önce,hachmeister+partner,"Bielefeld, Kuzey Ren-Vestfalya, Almanya",None,linkedin
4,Data Analyst für Datenmodellierung und Kundenb...,1 ay önce,Lurse,"Salzkotten, Kuzey Ren-Vestfalya, Almanya",None,linkedin


# Read Data 

In [38]:
jobs = pd.read_csv(f'{path}/{job_name2}-{time_}.csv')
df = jobs.copy()
df.head()

,job_title,publish,company_name,city,description,website
0,Data Scientist / Analyst - Web (m/w/d),vor 5 Tagen,Neue Westfälische GmbH & Co. KG,Bielefeld,Wir suchen nach einem Data Analyst/Data Scient...,stepstone
1,ERP Specialist / Business Data Analyst (m/w/d),vor 1 Woche,DAYTON PROGRESS GmbH,"Hamburg, Köln, Bielefeld, Hannover, Berlin, Le...",Aktuell sucht Dayton Progress einen ERP Specia...,stepstone
2,Data Analyst / Scientist (m/w/d),vor 1 Woche,eWolff GmbH,Bielefeld,Finde Deine Rolle und verstärke unser Team Dat...,stepstone
3,Consultant Data Analyst (m/w/d),vor 2 Tagen,Arvato Systems GmbH,Gütersloh,Und darum suchen wir dich als Consultant Data ...,stepstone
4,Data / Business Analyst (m/w/d),vor 1 Woche,eWolff GmbH,Bielefeld,Finde Deine Rolle und verstärke unser Team Dat...,stepstone


In [104]:
df = jobs.copy()

In [22]:
df.shape

(717, 6)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_title     717 non-null    object
 1   publish       712 non-null    object
 2   company_name  704 non-null    object
 3   city          717 non-null    object
 4   description   541 non-null    object
 5   website       717 non-null    object
dtypes: object(6)
memory usage: 33.7+ KB


## Dropn Null Values

In [105]:
df.isna().sum()

job_title         0
publish           5
company_name     13
city              0
description     176
website           0
dtype: int64

In [106]:
df.dropna(subset=['company_name'], inplace=True)
df.isna().sum()

job_title         0
publish           0
company_name      0
city              0
description     163
website           0
dtype: int64

## Drop Duplicated Values

In [125]:
df[df.duplicated(subset=['job_title', 'company_name', 'city', 'description'])].count()

job_title       5
publish         5
company_name    5
city            5
description     5
website         5
dtype: int64

In [128]:
df.drop_duplicates(subset=['job_title', 'company_name', 'city', 'description'], inplace=True)
df[df.duplicated(subset=['job_title', 'company_name', 'city', 'description'])].count()

job_title       0
publish         0
company_name    0
city            0
description     0
website         0
dtype: int64

In [130]:
df.shape

(699, 6)

In [16]:
## Modify Publish Column

In [96]:
def replace_column(column_name, old, new):
    df[column_name] = df[column_name].str.replace(old, new)

In [131]:
from datetime import timedelta
# print(d - datetime.timedelta(days=57)

def replace_column(column_name, old, new):
    df[column_name] = df[column_name].str.replace(old, new)

# Rakam Secer String icinden
#df.publish.str.replace('\D+', '')

# Fill NAN to no_info
df.publish = df.publish.str.lower()
df.publish.fillna('no_info', inplace=True)

# Fill stunde to today
time_ = datetime.today().strftime('%Y-%m-%d')
df.publish.loc[df.publish.str.contains('stunden')] = time_

# clean vor and önce
replace_column('publish', 'vor', '')
replace_column('publish', 'önce', '')

# strip
df.publish = df.publish.str.strip()

#df.publish.loc[df.publish.str.contains('tag')] = df.publis
df.publish.unique()

array(['5 tagen', '1 woche', '2 tagen', '3 tagen', '3 wochen', '4 tagen',
       '2 wochen', '1 tag', '4 wochen', '2022-11-07', '6 tagen',
       '1 monat', '29.10.2022', '27.10.2022', '24.10.2022', '26.10.2022',
       '04.10.2022', '30.10.2022', '19.10.2022', '26.06.2022', '1 hafta',
       '2 hafta', '1 gün', '1 ay', '4 gün', '3 hafta', '3 ay', '4 hafta',
       '2 gün', '2 ay', '4 ay', '6 gün', '3 gün', '6 ay'], dtype=object)

In [98]:
df.groupby('company_name')['job_title'].count().reset_index().sort_values('job_title', ascending=False).head(25)

,company_name,job_title
180,NTT DATA Business Solutions AG,97
104,FORTIS IT-Services GmbH,24
263,adesso SE,17
78,DATAGROUP,13
205,Reply,12
27,Arvato Systems,11
238,Tourlane GmbH,11
199,PwC,11
234,Takeaway.com,9
28,Arvato Systems GmbH,9


In [103]:
df[df[['job_title', 'company_name', 'city', 'description']].duplicated()]

,job_title,publish,company_name,city,description,website
144,System Engineer Storage / NAS (m/w/d),1 woche,Arvato Systems GmbH,Gütersloh,Hohes Know-how über die ONTAP Versionen Cluste...,stepstone


In [99]:
df.columns

Index(['job_title', 'publish', 'company_name', 'city', 'description',
       'website'],
      dtype='object')